In [1]:
#import tensorflow as tf
import numpy as np
import math
import glob
import matplotlib

import matplotlib.pyplot as plt
from matplotlib.pyplot import plot,savefig
from dataset import *

from sklearn.datasets import load_iris
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split,cross_val_score, cross_validate
import mylib as ml2

from sklearn.svm import SVC
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap
 
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import pandas as pd
import plotly.figure_factory as ff

#setting offilne
plotly.offline.init_notebook_mode(connected=True)


/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
#------------------------read data----------------------------#
cutsize = 256
circle_num =2
expansion_data_number = 1000
noise_scales = 0.05
whether_expansion_data = 1
"""
dataset_file_name = 'dataset_' + str(circle_num) + '_' + str(cutsize) + '.npy'
label_file_name = 'label_' + str(circle_num) + '_' + str(cutsize) + '.npy'
"""
data_path = '/home/silver-bullet/newpaper/data/dataset/'
dataset, label = load_dataset(data_path, circle_num=circle_num, cutsize=cutsize) 
#data_path = '/home/silver-bullet/newpaper/data/CWRUdataset'
#dataset, label = load_CWRU_data(data_path)

if whether_expansion_data == 1:
    dataset, label = expansion_and_add_noise(dataset,label,exnumber=expansion_data_number, noise_scales=noise_scales)


x_number = dataset.shape[0]

feature_data = feature_extractor2(dataset)
print('feature_data.shape: ', feature_data.shape)

x = feature_data.reshape(x_number,-1)
print('x.shape: ',x.shape)
y = label.reshape(x_number,)
print('y.shape: ', y.shape)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=56)

print('X_train.shape: ', X_train.shape)
print('X_test.shape: ',X_test.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

dataset shape :  (3710, 1280, 3)
label shape :  (3710, 1)
(647, 1280, 3)
(810, 1280, 3)
(761, 1280, 3)
(756, 1280, 3)
(731, 1280, 3)
(1000, 1280, 3)
(1000, 1)
(1000, 1280, 3)
(1000, 1)
(1000, 1280, 3)
(1000, 1)
(1000, 1280, 3)
(1000, 1)
(1000, 1280, 3)
(1000, 1)
(5000, 1280, 3)
(5000, 1)
data_max_abs_x.shape:  (5000, 1, 3)
noise_data.shape:  (5000, 1280, 3)
after expansion: 
dataset shape :  (8710, 1280, 3)
label shape :  (8710, 1)
p1.shape:  (8710, 3)
p2.shape:  (8710, 3)
p3.shape:  (8710, 3)
p4.shape:  (8710, 3)
p5.shape:  (8710, 3)
p6.shape:  (8710, 3)
p7.shape:  (8710, 3)
p8.shape:  (8710, 3)
p9.shape:  (8710, 3)
feature_data.shape:  (8710, 9, 3)
x.shape:  (8710, 27)
y.shape:  (8710,)
X_train.shape:  (6968, 27)
X_test.shape:  (1742, 27)
y_train.shape:  (6968,)
y_test.shape:  (1742,)


In [3]:
#------------------------create and train svm----------------------------#
sc = StandardScaler()
sc.fit(X_train)

X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

X_combined_std = np.vstack((X_train_std,X_test_std))
y_combined_std = np.hstack((y_train,y_test))
print('X_train_std.shape: ',X_train_std.shape)

svm = SVC(kernel='linear',C=1.0,random_state= 0)
svm.fit(X_train_std,y_train)

predict_y = svm.predict(X_test_std)

print('predict_x.shape: ', predict_y.shape)

ml2.evaluate_model2(predict_y, y_test)

# 计算准确率
cnt1 = 0
cnt2 = 0
for i in range(len(y_test)):
    if predict_y[i] == y_test[i]:
        cnt1 += 1
    else:
        cnt2 += 1

print("Total Accuracy: %.2f %% " % (100 * cnt1 / (cnt1 + cnt2)))


X_train_std.shape:  (6968, 27)
predict_x.shape:  (1742,)
normal result:
tp: 326  tn: 0  fp: 2  fn: 1414
tp/(tp+tn):  1.0
tp/(tp+fp):  0.9939024390243902
inner_ring result:
tp: 332  tn: 28  fp: 47  fn: 1335
tp/(tp+tn):  0.9222222222222223
tp/(tp+fp):  0.8759894459102903
outer_ring result:
tp: 355  tn: 18  fp: 22  fn: 1347
tp/(tp+tn):  0.9517426273458445
tp/(tp+fp):  0.9416445623342176
roller result:
tp: 316  tn: 20  fp: 32  fn: 1374
tp/(tp+tn):  0.9404761904761905
tp/(tp+fp):  0.9080459770114943
joint result:
tp: 282  tn: 65  fp: 28  fn: 1367
tp/(tp+tn):  0.8126801152737753
tp/(tp+fp):  0.9096774193548387
Total Accuracy: 92.48 % 


In [4]:
#-----------k fold----------------#

#scoring = ['accuracy','recalling']
scoring = ['precision_macro', 'recall_macro']
#scores = cross_validate(svm, X_test_std, y_test, scoring=scoring,cv=10, return_train_score=True)

scores = cross_validate(svm, X_train_std, y_train, scoring=scoring,cv=10, return_train_score=True)
sorted(scores.keys())
print('test result:')
print(scores)


#plot
init_notebook_mode(connected=True)
scores_df = pd.DataFrame(scores)

table_header = ['test_recall_macro', 'train_recall_macro','fit_time', 'train_precision_macro','test_precision_macro']

scores_df

test result:
{'fit_time': array([0.36968637, 0.36482763, 0.36088371, 0.35472512, 0.3638351 ,
       0.34857678, 0.35858607, 0.35883141, 0.35407567, 0.3627429 ]), 'test_recall_macro': array([0.93052547, 0.93312274, 0.93580505, 0.92898355, 0.9205949 ,
       0.91603225, 0.92066157, 0.91437132, 0.91425778, 0.92691499]), 'score_time': array([0.05643225, 0.05518889, 0.05635405, 0.0540998 , 0.05538583,
       0.05614901, 0.05485296, 0.05507565, 0.05241799, 0.05513501]), 'test_precision_macro': array([0.93109544, 0.93397902, 0.93707862, 0.92881623, 0.92179755,
       0.9150452 , 0.92067941, 0.91605775, 0.91510885, 0.92810047]), 'train_precision_macro': array([0.92806889, 0.92703972, 0.92655998, 0.92879645, 0.92964771,
       0.92935104, 0.92837385, 0.93007086, 0.92857744, 0.9271503 ]), 'train_recall_macro': array([0.92784824, 0.92691433, 0.92647842, 0.9286843 , 0.92959107,
       0.92921997, 0.9283758 , 0.92979305, 0.92820649, 0.92712482])}


,fit_time,score_time,test_precision_macro,test_recall_macro,train_precision_macro,train_recall_macro
0,0.369686,0.056432,0.931095,0.930525,0.928069,0.927848
1,0.364828,0.055189,0.933979,0.933123,0.927040,0.926914
2,0.360884,0.056354,0.937079,0.935805,0.926560,0.926478
3,0.354725,0.054100,0.928816,0.928984,0.928796,0.928684
4,0.363835,0.055386,0.921798,0.920595,0.929648,0.929591
5,0.348577,0.056149,0.915045,0.916032,0.929351,0.929220
6,0.358586,0.054853,0.920679,0.920662,0.928374,0.928376
7,0.358831,0.055076,0.916058,0.914371,0.930071,0.929793
8,0.354076,0.052418,0.915109,0.914258,0.928577,0.928206
9,0.362743,0.055135,0.928100,0.926915,0.927150,0.927125
